In [1]:
%load_ext autoreload
%autoreload 2

# organize the imports
import opentrons.execute
from opentrons import types 
import time
import numpy as np
import opentrons
from  opentrons.types import Location, Point

from opentrons_helper import * 

In [74]:
from imjoy_rpc import api
class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        # for regular plugin
        # p = await api.getPlugin(my_plugin_source)

        # or for window plugin
        # await api.createWindow(src=my_plugin_source)
        await api.showDialog(src=my_plugin_source)

api.export(ImJoyPlugin())

<IPython.core.display.Javascript object>

<_GatheringFuture pending>

In [2]:
'''
Connect to the microscope 
'''

from microscope_client import MicroscopeClient 
microscope = MicroscopeClient(host="21.3.2.3", port=5000)
#!pip3 install zeroconf
#!pip3 install requests


In [3]:
# Connect to the opentrons robot
protocol = opentrons.execute.get_protocol_api('2.9')
protocol.home()

# setup the pipets
pipette = protocol.load_instrument('p300_single_gen2', 'right')
pipette_8 = protocol.load_instrument('p300_multi', 'left') # causes errors -> don't use two pipettes!!


In [4]:
try:
    pipette.drop_off()
except:
    print("No pipette attached")
pipette.home()

No pipette attached


<InstrumentContext: p300_single_v2.1 in RIGHT>

In [5]:
# setup the labware in the trays
#plate_microscope = protocol.load_labware('corning_96_wellplate_360ul_flat', location='5')
#plate_reagents = protocol.load_labware('corning_24_wellplate_3.4ml_flat', location='9')
plate_reagents = protocol.load_labware('opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap', location='10')


# Move light to the sample 
tiprack = protocol.load_labware('opentrons_96_tiprack_300ul',location='11')

# sample plate on microscope
# offset of the well plate in mm
plate_sample = protocol.load_labware('corning_96_wellplate_360ul_flat', location='3')

In [6]:
from opentrons.types import Location, Point

# set speed
#protocol.max_speeds['X'] = 200 # default = 400
#protocol.max_speeds['Y'] = 200 # default = 300

# ######
# 0 4 8
# 1 5 9
# 2 6 10 
# 3 7 11

# wells with reagents
i_ab_primary = 0
i_ab_secondary = 1
i_pbs = 2
i_bsa = 3


# define timig
t_bsa = 15
t_primary = 60
t_secondary = 60
t_fluo = 30*0

# move it down since opentrons expects the reagents to be furthter away from the ground
offset_z_reagents = -65

# define locations 
minimum_z_height = 150 # minimum height for arcs driven by the robot

# compute the positions
position_trash =  (365,350, 100)

position_idle =  (100,100, 150)#

In [7]:
#
move_to_coord(pipette, position_trash, offset=(0,0,0), minimum_z_height=minimum_z_height)

Please add a pipette first!
Moving to: (365, 350, 100)


In [39]:
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

Moving to: (100, 100, 150)


In [68]:
from opentrons_client import OpentronsClient 
ot2 = OpentronsClient(host="21.3.2.5", port=31950)
ot2.toggle_light(True)
#ipot2.pipette_home(location="right")
pos = ot2.positions()

HTTPError: 422 Client Error: Unprocessable Entity for url: http://21.3.2.5:31950/robot/positions

AttributeError: 'OpentronsClient' object has no attribute 'positions'

In [9]:
i_pipette  = 0

# Calibrate

In [10]:
'''
calibrate tiprack
'''
#pitty, but moving the xy coordinates doesn't work
print(i_pipette)
offset_pipette_rack=(1,7,0)
tiprack.set_calibration(Point(*offset_pipette_rack))

i_pipette=pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack=offset_pipette_rack)
print(type(i_pipette))

0
My pipette number: 0and type: <class 'int'>
<class 'int'>


In [41]:
'''
calibrate illumination poisiton
'''

#protocol.home()
#position_idle =  (100,300, 150)#
#move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

position_sample_light = (253, 99, 202)
offset_x = 35000 # 1290+0*9000
offset_y = 43000# -1400+4*9000
offset_z = microscope.position["z"]
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
microscope.set_laser_led(0,0)
microscope.move((offset_x,offset_y,offset_z),absolute=True)
autofocus_dz = 500
autofocus_Nz = 7
is_autofocus = False
#offset_z = microscope.autofocus_coarse(dz=autofocus_dz, nz=autofocus_Nz)
microscope.set_laser_led(0,0)

Please add a pipette first!
Moving to: (253, 99, 202)


In [38]:
'''
Calibrate Wellposition for sample coordinates (96well plate)
pipette should dip in the wells on the microscope

'''
#pipette.home()
offset_z = microscope.position["z"]
home_position_microscope=(0,0,offset_z)
microscope.move((home_position_microscope))


#i_pipette=pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack=offset_pipette_rack)


sample_position_15welplate =  (286+15, 115-4, 130.885) 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)


Moving to: (301, 111, 130.885)


In [13]:
'''
calibrate reagetn slots
'''
#pipette.home()

offset_x_reagents = -4
offset_y_reagents = 10
offset_z_reagents = -55 # smaller number means: go up
for i_pos in range(4):
    #i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, -8)
    position_reagent_i = np.array(plate_reagents.wells()[i_pos].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)

# move out of the well again
move_to_coord(pipette, position_reagent_i, offset=(0,0,20), minimum_z_height=minimum_z_height)

    

Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (14.21, 337.65, 5.949999999999996)
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (14.21, 299.09, 5.949999999999996)
Moving to: (18.21, 289.09, 80.94999999999999)


In [14]:
if(0): #test laser 
    microscope.set_laser_led(0,1)
    autofocus_dz = 200
    autofocus_Nz = 7
    offset_z = autofocus_coarse(dz=autofocus_dz, nz=autofocus_Nz)
    microscope.set_laser_led(0,0)
    
    # take BF image

    move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
    time.sleep(1) # debounce
    microscope.move((offset_x,offset_y,offset_z),absolute=True)
    microscope.capture_image_to_disk()

    # take Fluo image
    move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)
    time.sleep(1) # debounce
    microscope.set_laser_led(1,1)
    microscope.capture_image_to_disk()
    time.sleep(1)
    microscope.set_laser_led(0,0)

In [15]:
# initialize strins


from datetime import datetime
timestamp = datetime.now().strftime("%Y_%m_%d-%I-%M-%S_%p")
i_experiment = 0


# Do a time-lapse for an hour

# Add Blocking Buffer

 we assume the sample is fixed and permeabilised and needs blocking fore primary/secondary antibody labelling

In [17]:

# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

print(type(i_pipette))
V_aspirate = 40
        
# first go to sample and pickup liquid 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.aspirate(100)

# 3. Trash the liquid
move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)
pipette.dispense()
                

# 4- Pickup BSa and bring it to sample         
position_reagent_i = np.array(plate_reagents.wells()[i_bsa].center()._point)
move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
pipette.aspirate(V_aspirate)

# 5. Go to sample and add BSA 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.dispense()


# go to idle positoin and rest
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

# wait for 15 minutes 
#time.sleep(15*60)

My pipette number: 1and type: <class 'int'>
<class 'int'>
Moving to: (301, 111, 130.885)
Moving to: (361, 360, 55)
Moving to: (14.21, 299.09, 5.949999999999996)
Moving to: (301, 111, 130.885)
Moving to: (100, 100, 150)


# Do timelapse for 15 minutes 

In [25]:
# Microscope coordinates
autofocus_dz = 500
autofocus_Nz= 9
mytime = time.time()
t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list
Nx=5
Ny=5
well_to_well_steps= 500
t_duration = 15 # how long does the experiment preform?
t_period = 1 # min - how often should the expimrent be carried out?

while (time.time()-mytime)<(t_bsa*60):
    
    # perform a testing well scan by moving robot to light position and do a whole plate scan
    move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
    focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
            Nx, Ny, well_to_well_steps,
            autofocus_dz, autofocus_Nz,
                name_experiment="wellscan_afterbsa_"+timestamp,
                focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
                is_autofocus_fine = False, 
                is_autofocus_fast = False,
                I_laser=0, I_led=1)
    i_experiment += 1
    
    image_stack
    

#n_cells = imjoy.plugin("Countcells", image_stack)
#new_protocol = decide(n_cells)

Please add a pipette first!
Moving to: (253, 99, 202)
Start moving to the position
Start scan
Move microscope to (XY): 35000/51000
offset_z:-7259
Move microscope to (XY): 35500/51000
offset_z:-7259
Move microscope to (XY): 36000/51000
offset_z:-7259
Move microscope to (XY): 36500/51000
offset_z:-7259
Move microscope to (XY): 37000/51000
offset_z:-7259
Move microscope to (XY): 35000/51500
offset_z:-7259
Move microscope to (XY): 35500/51500
offset_z:-7259
Move microscope to (XY): 36000/51500
offset_z:-7259
Move microscope to (XY): 36500/51500
offset_z:-7259
Move microscope to (XY): 37000/51500
offset_z:-7259
Move microscope to (XY): 35000/52000
offset_z:-7259
Move microscope to (XY): 35500/52000
offset_z:-7259
Move microscope to (XY): 36000/52000
offset_z:-7259
Move microscope to (XY): 36500/52000
offset_z:-7259
Move microscope to (XY): 37000/52000
offset_z:-7259
Move microscope to (XY): 35000/52500
offset_z:-7259
Move microscope to (XY): 35500/52500
offset_z:-7259
Move microscope to (XY

# Add Primary

In [26]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)


print(type(i_pipette))
V_aspirate = 40
        
# first go to sample and pickup liquid 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.aspirate()

# 3. Trash the liquid
move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)
pipette.dispense()
                
# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

# 4- Pickup Primary and bring it to sample         
position_reagent_i = np.array(plate_reagents.wells()[i_ab_primary].center()._point)
move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
pipette.aspirate(V_aspirate)

# 5. Go to sample and add BSA 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.dispense()


# go to idle positoin and rest
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)


<class 'int'>
Moving to: (301, 111, 130.885)
Moving to: (361, 360, 55)
My pipette number: 2and type: <class 'int'>
Moving to: (14.21, 356.93, 5.949999999999996)
Moving to: (301, 111, 130.885)
Moving to: (100, 100, 150)


# Do timelapse for an hour

In [27]:
# Microscope coordinates

mytime = time.time()
t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list
Nx=5
Ny=5
well_to_well_steps= 500
t_duration = 60 # how long does the experiment preform?
t_period = 1 # min - how often should the expimrent be carried out?

while (time.time()-mytime)<(t_primary*60):
    
    # perform a testing well scan by moving robot to light position and do a whole plate scan
    move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
    focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
            Nx, Ny, well_to_well_steps,
            autofocus_dz, autofocus_Nz,
                name_experiment="wellscan_afterprimary_"+timestamp,
                focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
                is_autofocus_fine = False, 
                is_autofocus_fast = False,
                I_laser=0, I_led=1)
    i_experiment += 1

Please add a pipette first!
Moving to: (253, 99, 202)
Start moving to the position
Start scan
Move microscope to (XY): 35000/51000
offset_z:-7153
Move microscope to (XY): 35500/51000
offset_z:-7153
Move microscope to (XY): 36000/51000
offset_z:-7153
Move microscope to (XY): 36500/51000
offset_z:-7153
Move microscope to (XY): 37000/51000
offset_z:-7153
Move microscope to (XY): 35000/51500
offset_z:-7153
Move microscope to (XY): 35500/51500
offset_z:-7153
Move microscope to (XY): 36000/51500
offset_z:-7153
Move microscope to (XY): 36500/51500
offset_z:-7153
Move microscope to (XY): 37000/51500
offset_z:-7153
Move microscope to (XY): 35000/52000
offset_z:-7153
Move microscope to (XY): 35500/52000
offset_z:-7153
Move microscope to (XY): 36000/52000
offset_z:-7153
Move microscope to (XY): 36500/52000
offset_z:-7153
Move microscope to (XY): 37000/52000
offset_z:-7153
Move microscope to (XY): 35000/52500
offset_z:-7153
Move microscope to (XY): 35500/52500
offset_z:-7153
Move microscope to (XY

# Wash sample 3x 

In [28]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)


print(type(i_pipette))
V_aspirate = 40

for _ in range(3):
    # first go to sample and pickup liquid 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.aspirate()

    # 3. Trash the liquid
    move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)
    pipette.dispense()

    # 1. Pickup new pipette
    i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

    # 4- Pickup Primary and bring it to sample         
    position_reagent_i = np.array(plate_reagents.wells()[i_pbs].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate)

    # 5. Go to sample and add PBS 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()


# go to idle positoin and rest
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

    # wait for 15 minutes 
    #time.sleep(15*60)

<class 'int'>
Moving to: (301, 111, 130.885)
Moving to: (361, 360, 55)
My pipette number: 3and type: <class 'int'>
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (301, 111, 130.885)
Moving to: (301, 111, 130.885)
Moving to: (361, 360, 55)
My pipette number: 4and type: <class 'int'>
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (301, 111, 130.885)
Moving to: (301, 111, 130.885)
Moving to: (361, 360, 55)
My pipette number: 5and type: <class 'int'>
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (301, 111, 130.885)
Moving to: (100, 100, 150)


# Add Secondary 

In [29]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)


print(type(i_pipette))
V_aspirate = 40
        
# first go to sample and pickup liquid 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.aspirate()

# 3. Trash the liquid
move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)
pipette.dispense()

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)
               

# 4- Pickup Primary and bring it to sample         
position_reagent_i = np.array(plate_reagents.wells()[i_ab_secondary].center()._point)
move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
pipette.aspirate(V_aspirate)

# 5. Go to sample and add Secondary 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.dispense()

# go to idle positoin and rest
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

# wait for 15 minutes 
#time.sleep(t_secondary*60)

<class 'int'>
Moving to: (301, 111, 130.885)
Moving to: (361, 360, 55)
My pipette number: 6and type: <class 'int'>
Moving to: (14.21, 337.65, 5.949999999999996)
Moving to: (301, 111, 130.885)
Moving to: (100, 100, 150)


In [30]:
# Microscope coordinates

mytime = time.time()
t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list
Nx=5
Ny=5
well_to_well_steps= 500
t_duration = 60 # how long does the experiment preform?
t_period = 1 # min - how often should the expimrent be carried out?

##
mytime = -1
while (time.time()-mytime)<(t_secondary*60):
    mytime = time.time()
    if time.time()-mytime2>t_period:
        mytime2=time.time()
    

        # perform a testing well scan by moving robot to light position and do a whole plate scan
        move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
        focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
                Nx, Ny, well_to_well_steps,
                autofocus_dz, autofocus_Nz,
                    name_experiment="wellscan_aftersecondary_"+timestamp,
                    focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
                    is_autofocus_fine = False, 
                    is_autofocus_fast = False,
                    I_laser=0, I_led=1)
        i_experiment += 1

# Wash 3x with PBS

In [31]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
#i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)
V_aspirate = 40

for _ in range(3):
    # first go to sample and pickup liquid 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.aspirate()

    # 3. Trash the liquid
    move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)
    pipette.dispense()


    # 4- Pickup Primary and bring it to sample         
    position_reagent_i = np.array(plate_reagents.wells()[i_pbs].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate)

    # 5. Go to sample and add PBS 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()


# go to idle positoin and rest
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

    # wait for 15 minutes 
    #time.sleep(15*60)

Moving to: (301, 111, 130.885)
Moving to: (361, 360, 55)
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (301, 111, 130.885)
Moving to: (301, 111, 130.885)
Moving to: (361, 360, 55)
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (301, 111, 130.885)
Moving to: (301, 111, 130.885)
Moving to: (361, 360, 55)
Moving to: (14.21, 318.37, 5.949999999999996)
Moving to: (301, 111, 130.885)
Moving to: (100, 100, 150)


# Do timelapse

In [32]:
# Microscope coordinates

mytime = time.time()
t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list
Nx=5
Ny=5
well_to_well_steps= 500
t_duration = 60 # how long does the experiment preform?
t_period = 1 # min - how often should the expimrent be carried out?

##
mytime = -1
while (time.time()-mytime)<(t_secondary*60):
    mytime = time.time()
    if time.time()-mytime2>t_period:
        mytime2=time.time()
    

        # perform a testing well scan by moving robot to light position and do a whole plate scan
        move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
        focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
                Nx, Ny, well_to_well_steps,
                autofocus_dz, autofocus_Nz,
                    name_experiment="wellscan_finished_"+timestamp,
                    focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
                    is_autofocus_fine = False, 
                    is_autofocus_fast = False,
                    I_laser=0, I_led=1)
        i_experiment += 1

In [33]:
# Microscope coordinates

mytime = time.time()
t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list
Nx=5
Ny=5
well_to_well_steps= 500
t_duration = 60 # how long does the experiment preform?
t_period = 1 # min - how often should the expimrent be carried out?

##
mytime = -1
while (time.time()-mytime)<(t_fluo*60):
    mytime = time.time()
    if time.time()-mytime2>t_period:
        mytime2=time.time()

        # perform a testing well scan by moving robot to light position and do a whole plate scan
        move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)
        focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
                Nx, Ny, well_to_well_steps,
                autofocus_dz, autofocus_Nz,
                    name_experiment="wellscan_aftersecondary_"+timestamp,
                    focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
                    is_autofocus_fine = False, 
                    is_autofocus_fast = False,
                    I_laser=0, I_led=1)
        i_experiment += 1

In [73]:
## ImJoy Plugin
from IPython.display import HTML
my_plugin_source = HTML('''
<docs lang="markdown">
[TODO: write documentation for this plugin.]
</docs>

<config lang="json">
{
  "name": "Untitled Plugin",
  "type": "window",
  "tags": [],
  "ui": "",
  "version": "0.1.0",
  "cover": "",
  "description": "[TODO: describe this plugin with one sentence.]",
  "icon": "extension",
  "inputs": null,
  "outputs": null,
  "api_version": "0.1.8",
  "env": "",
  "permissions": [],
  "requirements": [],
  "dependencies": [],
  "defaults": {"w": 20, "h": 10}
}
</config>

<script lang="javascript">
class ImJoyPlugin {
  async setup() {
    api.log('initialized')
  }

  async run(ctx) {

  }
}

api.export(new ImJoyPlugin())
</script>

<window lang="html">
  <div>
    <p>
    This is an awesome project, where you have open-source hardware, software, minds and many genius people in one place. Wow! 
        ImJoy, Opentrons, openUC2 and Openflexure all in one place! 
    </p>
  </div>
</window>

<style lang="css">

</style>
''')